In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
mnet = MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mnet

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

dataset, metadata = tfds.load('deep_weeds', as_supervised=True, with_info=True)
train_dataset = dataset['train']

class_names = metadata.features['label'].names
print("Class names: {}".format(class_names))

In [ ]:
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

def resize(images, labels):
    images = tf.image.resize(images, size=(224, 224))

    return images, labels

def transform_images(images, labels):
    images, labels = resize(images, labels)
    images, labels = normalize(images, labels)

    return images, labels

train_dataset =  train_dataset.map(transform_images)
train_dataset =  train_dataset.cache()

In [ ]:
for image, labels in train_dataset.take(1):
    break
pred = mnet(tf.expand_dims(image, 0))
pred.shape

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
mnet.trainable = False
gap = GlobalAveragePooling2D()
dropout = Dropout(rate=0.25)
fc1 = Dense(units=len(class_names), activation=tf.nn.softmax)
layers =[
    mnet,
    gap,
    dropout,
    fc1,
]
model = tf.keras.models.Sequential(layers)

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
import numpy as np
BATCH_SIZE = 32
num_train_examples = metadata.splits['train'].num_examples
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(BATCH_SIZE)

model.fit(train_dataset, epochs=10, steps_per_epoch=np.ceil(num_train_examples/BATCH_SIZE))